# Making maps
![all the suburbs in sydney](all_burbs.png)

We're going to get some shapefiles from data.gov.au, and then we're going to draw some maps

You'll need some prequisite libraries:
```
pip install pandas shapely fiona descartes pyproj geopandas
```
You may have some already but pip will be cool with that

In [15]:
import os
import geopandas as gp
import shapely
import fiona
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd



In [16]:
%matplotlib inline
plt.rcParams['figure.figsize'] = (20, 10)

#### Shapefiles are the format that a lot of GIS tools use.

#### GIS means Geographic Information System

pronounced: _gee eye ess_

not pronounced: _jizz_

In [73]:
print(os.getcwd())
print(os.listdir("."))
fp = 'QLD_LOCALITY_POLYGON_shp.shp'
print(fp in os.listdir("."))
fp_exists = os.path.exists(fp)
print(fp_exists)
if fp_exists:
    print("I exist")
    burbs = gp.GeoDataFrame.from_file(fp)
else:
    raise Exception("no burbs data")
burbs.head(2)

/Users/joelnorman/Documents/1161/me/maps
['.DS_Store', 'QLD_LOCALITY_shp.dbf', 'QLD_LOCALITY_POLYGON_shp.prj', 'QLD_LOCALITY_POLYGON_shp.qpj', 'QLD_LOCALITY_POLYGON_shp.dbf', 'maps.ipynb', 'QLD map', 'QLD_LOCALITY_POLYGON_shp.shx', 'all_burbs.png', 'QLD_LOCALITY_POLYGON_shp.cpg', 'QLD_LOCALITY_POLYGON_shp.shp']
True
True
I exist


,LC_PLY_PID,DT_CREATE,DT_RETIRE,LOC_PID,QLD_LOCALI,QLD_LOCA_1,QLD_LOCA_2,QLD_LOCA_3,QLD_LOCA_4,QLD_LOCA_5,QLD_LOCA_6,QLD_LOCA_7,geometry
0,67839,2018-09-13,None,QLD1991,2016-08-10,None,MOUNT BERRYMAN,None,None,G,None,3,"POLYGON ((152.30132 -27.75523, 152.30122 -27.7..."
1,64808,2017-06-14,None,QLD2092,2017-08-10,None,MOUNTAIN CREEK,None,None,G,None,3,"POLYGON ((153.10559 -26.71514, 153.10323 -26.7..."


In [20]:
#  burbs.geometry.plot()
#  could plot, if you had a really fast computer!

In [21]:
a = burbs.iloc[0]
print(a)
a.geometry

NameError: name 'burbs' is not defined

In [22]:
def add_centroid(row):
    return row.geometry.centroid

burbs["centroid"] = burbs.apply(add_centroid, axis=1)

NameError: name 'burbs' is not defined

In [23]:
a = burbs.iloc[0]
print(a.centroid)
a.centroid

NameError: name 'burbs' is not defined

In [24]:
right_here = shapely.geometry.point.Point(151.2299732, -33.9178754)
burbs["distance_from_UNSW"] = burbs.geometry.distance(right_here)

NameError: name 'burbs' is not defined

In [25]:
burbs.distance_from_UNSW.hist(bins=50);

NameError: name 'burbs' is not defined

This gives distance in whole numbers of lat long I think, i.e. degrees!

Not really sure what to do with that, but let's get rid of everything above 0.2 and see what we can plot:

In [26]:
close_burbs = burbs[burbs.distance_from_UNSW<0.2]
close_burbs.plot();

NameError: name 'burbs' is not defined

In [27]:
close_burbs.geometry.convex_hull.plot();

NameError: name 'close_burbs' is not defined

In [28]:
close_burbs.geometry.envelope.plot();

NameError: name 'close_burbs' is not defined

![](http://www.personal.kent.edu/~rmuhamma/Compgeometry/MyCG/Gifs-CompGeometry/ch2.gif)
Convex hulls are a useful aproximation if you want to do fast calculations.

Bounding boxes are even cheaper.

What about putting labels on the map?

In [29]:
really_close_burbs = burbs[burbs.distance_from_UNSW<0.03]
really_close_burbs.plot()

for idx, row in really_close_burbs.iterrows():
    plt.annotate(text=row.NSW_LOCA_2, 
                 xy=tuple(row.centroid.coords)[0],
                 horizontalalignment='center')
plt.title("Some pretty dodgy suburb names on a map");

NameError: name 'burbs' is not defined

We often want to be able to tell if a point is in a suburb or not. We'd do that with a polygon inclusion test.

![](http://www.geeksforgeeks.org/wp-content/uploads/polygon31.png)
[How to check if a given point lies inside or outside a polygon?](http://www.geeksforgeeks.org/how-to-check-if-a-given-point-lies-inside-a-polygon/)

Luckily we've got one built in!

In [30]:
print(right_here)
in_this_burb = None
for _, row in really_close_burbs.iterrows():
    if right_here.within(row.geometry):
        in_this_burb = row
        
in_this_burb

POINT (151.2299732 -33.9178754)


NameError: name 'really_close_burbs' is not defined

This might actually be a case where a `lambda` is a good idea:

In [31]:
in_this_burb = really_close_burbs[really_close_burbs.apply(lambda x: right_here.within(x.geometry) , axis=1)]
in_this_burb

NameError: name 'really_close_burbs' is not defined

How about colouring suburbs according to some kind of scalar value?

[That's called a _chloropleth_](http://geopandas.readthedocs.io/en/latest/mapping.html?highlight=color#chloropleth-maps)

In [32]:
really_close_burbs.plot(column='distance_from_UNSW', cmap='cool');

NameError: name 'really_close_burbs' is not defined

I don't think that could be any easier!

You can see the list of [colour scale options here](http://matplotlib.org/users/colormaps.html)

## Handy links

* [Geopandas docs](http://geopandas.readthedocs.io)
* [Geopandas on GitHib](https://github.com/geopandas/geopandas)
* [Another map making tutorial](http://sensitivecities.com/so-youd-like-to-make-a-map-using-python-EN.html)
* [MatPlotLib colour scale options](http://matplotlib.org/users/colormaps.html)

I'd love to hear if you come across any other useful things!